In [1]:
import pandas as pd

In [18]:
DELAYS = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
DATES = ['YEAR', 'MONTH', 'DAY_OF_MONTH']
AIRLINE = 'AIRLINE_NAME'
ORIGIN = ['ORIGIN_AIRPORT_NAME', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM']
DEST = ['DEST_AIRPORT_NAME', 'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEST_STATE_NM']
DEPART = ['CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY_NEW', 'TAXI_OUT', 'WHEELS_OFF']
ARRIVAL = ['CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY_NEW', 'TAXI_IN', 'WHEELS_ON']
FLIGHT = ['CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE']

In [3]:
df = pd.read_csv('/Users/ykamoji/Documents/Semester4/COMPSCI_571/Project/Data/airlines_delay_data.csv')

## Sanity Check

In [5]:
df.head(5)

,YEAR,MONTH,DAY_OF_MONTH,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_NAME,DEST_CITY_NAME,...,WHEELS_ON,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2023,12,1,Endeavor Air Inc.,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...","Atlanta, GA",GA,Georgia,"Lafayette, LA: Lafayette Regional Paul Fournet...","Lafayette, LA",...,1704,106.0,106.0,86.0,503.0,30,0,0,0,0
1,2023,12,1,Endeavor Air Inc.,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...","Atlanta, GA",GA,Georgia,"Moline, IL: Quad Cities International","Moline, IL",...,1648,119.0,115.0,94.0,633.0,28,0,0,0,0
2,2023,12,1,Endeavor Air Inc.,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...","Atlanta, GA",GA,Georgia,"Roanoke, VA: Roanoke Blacksburg Regional","Roanoke, VA",...,1927,91.0,74.0,49.0,357.0,17,0,0,0,0
3,2023,12,1,Endeavor Air Inc.,"Austin, TX: Austin - Bergstrom International","Austin, TX",TX,Texas,"Raleigh/Durham, NC: Raleigh-Durham International","Raleigh/Durham, NC",...,1658,170.0,151.0,133.0,1162.0,0,0,0,0,202
4,2023,12,1,Endeavor Air Inc.,"Buffalo, NY: Buffalo Niagara International","Buffalo, NY",NY,New York,"New York, NY: John F. Kennedy International","New York, NY",...,1236,100.0,93.0,66.0,301.0,0,0,0,0,37


In [37]:
df[DELAYS].isnull().sum()

CARRIER_DELAY          0
WEATHER_DELAY          0
NAS_DELAY              0
SECURITY_DELAY         0
LATE_AIRCRAFT_DELAY    0
dtype: int64

In [32]:
df[DATES].isnull().sum()

YEAR            0
MONTH           0
DAY_OF_MONTH    0
dtype: int64

In [38]:
df[AIRLINE].isnull().sum()

0

In [39]:
df[ORIGIN].isnull().sum()

ORIGIN_AIRPORT_NAME    0
ORIGIN_CITY_NAME       0
ORIGIN_STATE_ABR       0
ORIGIN_STATE_NM        0
dtype: int64

In [40]:
df[DEST].isnull().sum()

DEST_AIRPORT_NAME    0
DEST_CITY_NAME       0
DEST_STATE_ABR       0
DEST_STATE_NM        0
dtype: int64

In [41]:
df[DEPART].isnull().sum()

CRS_DEP_TIME     0
DEP_TIME         0
DEP_DELAY_NEW    0
TAXI_OUT         0
WHEELS_OFF       0
dtype: int64

In [42]:
df[ARRIVAL].isnull().sum()

CRS_ARR_TIME     0
ARR_TIME         0
ARR_DELAY_NEW    0
TAXI_IN          0
WHEELS_ON        0
dtype: int64

In [43]:
df[FLIGHT].isnull().sum()

CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
dtype: int64

### Geo map calculations

In [70]:
df[['ORIGIN_STATE_ABR']+DELAYS].groupby('ORIGIN_STATE_ABR')[DELAYS].agg(['sum','count']).head(10)

CARRIER_DELAY                   WEATHER_DELAY              \
                           sum   count min   max           sum   count min   
ORIGIN_STATE_ABR                                                             
AK                      153735    7418   0  2225         26436    7418   0   
AL                      199712    7249   0  2540         54443    7249   0   
AR                      155461    5947   0  2672         53852    5947   0   
AZ                     1004549   40941   0  2086         77917   40941   0   
CA                     2999951  140027   0  3300        301601  140027   0   
CO                     1793852   79250   0  2012        317524   79250   0   
CT                      108660    4371   0  1685         14310    4371   0   
FL                     3393906  147057   0  2868        604057  147057   0   
GA                     2079186   69550   0  2353        236369   69550   0   
HI                      365581   15900   0  1549         11555   15900   0   

                       NAS_DELAY                   SECURITY_DELAY              \
                   max       sum   count min   max            sum   count min   
ORIGIN_STATE_ABR                                                                
AK                 627     60724    7418   0   192           4676    7418   0   
AL                1590     83725    7249   0   446            595    7249   0   
AR                1243     79173    5947   0   875           1049    5947   0   
AZ                1479    393784   40941   0   947           4131   40941   0   
CA                1487   1869959  140027   0  1501          14022  140027   0   
CO                1479    920878   79250   0   991           9643   79250   0   
CT                1127     68256    4371   0  1059            265    4371   0   
FL                1298   2187986  147057   0  1519          40176  147057   0   
GA                1318    890882   69550   0  1243           4464   69550   0   
HI                 578     62750   15900   0   501           3608   15900   0   

                      LATE_AIRCRAFT_DELAY                    
                  max                 sum   count min   max  
ORIGIN_STATE_ABR                                             
AK                 74              195206    7418   0   965  
AL                 84              242963    7249   0  1457  
AR                150              189425    5947   0  1206  
AZ                103             1131757   40941   0  1456  
CA                276             3942297  140027   0  2506  
CO                651             2137814   79250   0  1516  
CT                 33              160726    4371   0  1319  
FL                430             4848391  147057   0  2191  
GA                102             1367194   69550   0  2129  
HI                233              377353   15900   0  1398

In [72]:
df[['DEST_STATE_ABR'] + DELAYS].groupby('DEST_STATE_ABR')[DELAYS].agg(['sum', 'count']).head(10)

CARRIER_DELAY         WEATHER_DELAY         NAS_DELAY          \
                         sum   count           sum   count       sum   count   
DEST_STATE_ABR                                                                 
AK                    142745    8963         28006    8963     81133    8963   
AL                    188423    7838         40934    7838     74514    7838   
AR                    140563    6362         32210    6362     67612    6362   
AZ                   1193099   41446        146952   41446    334824   41446   
CA                   3193113  157021        431936  157021   2234339  157021   
CO                   1618022   65313        295753   65313    843093   65313   
CT                    135835    5329         15195    5329     50171    5329   
FL                   3439484  145196        443151  145196   2584317  145196   
GA                   1945688   62092        276166   62092    767336   62092   
HI                    441562   18557         21040   18557    117689   18557   

               SECURITY_DELAY         LATE_AIRCRAFT_DELAY          
                          sum   count                 sum   count  
DEST_STATE_ABR                                                     
AK                       4874    8963              194500    8963  
AL                        857    7838              223396    7838  
AR                        596    6362              183659    6362  
AZ                       6995   41446             1340082   41446  
CA                      20179  157021             3767677  157021  
CO                       5729   65313             1918849   65313  
CT                        459    5329              151518    5329  
FL                      20416  145196             4293278  145196  
GA                       7348   62092             1774742   62092  
HI                       2350   18557              336069   18557

In [74]:
df[df['CARRIER_DELAY'] == df['CARRIER_DELAY'].max()]

,YEAR,MONTH,DAY_OF_MONTH,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_NAME,DEST_CITY_NAME,...,WHEELS_ON,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
51133,2023,12,20,American Airlines Inc.,"Charleston, SC: Charleston AFB/International","Charleston, SC",SC,South Carolina,"Charlotte, NC: Charlotte Douglas International","Charlotte, NC",...,2124,67.0,76.0,42.0,168.0,3786,0,9,0,0


In [84]:
df.loc[df.groupby('ORIGIN_STATE_ABR')['LATE_AIRCRAFT_DELAY'].idxmax()]

,YEAR,MONTH,DAY_OF_MONTH,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_NAME,DEST_CITY_NAME,...,WHEELS_ON,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
1170239,2024,8,18,United Air Lines Inc.,"Anchorage, AK: Ted Stevens Anchorage Internati...","Anchorage, AK",AK,Alaska,"Newark, NJ: Newark Liberty International","Newark, NJ",...,7,416.0,415.0,392.0,3370.0,43,0,0,0,965
1159472,2024,8,16,PSA Airlines Inc.,"Birmingham, AL: Birmingham-Shuttlesworth Inter...","Birmingham, AL",AL,Alabama,"Philadelphia, PA: Philadelphia International","Philadelphia, PA",...,1703,138.0,128.0,107.0,772.0,9,0,0,0,1457
258578,2024,2,11,SkyWest Airlines Inc.,"Little Rock, AR: Bill and Hillary Clinton Nat ...","Little Rock, AR",AR,Arkansas,"Chicago, IL: Chicago O'Hare International","Chicago, IL",...,925,121.0,112.0,80.0,552.0,0,0,0,0,1206
871041,2024,6,29,American Airlines Inc.,"Phoenix, AZ: Phoenix Sky Harbor International","Phoenix, AZ",AZ,Arizona,"Columbus, OH: John Glenn Columbus International","Columbus, OH",...,2339,222.0,214.0,194.0,1670.0,23,0,0,0,1456
418605,2024,3,26,American Airlines Inc.,"Los Angeles, CA: Los Angeles International","Los Angeles, CA",CA,California,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...","Dallas/Fort Worth, TX",...,201,178.0,165.0,141.0,1235.0,121,0,0,0,2506
322659,2024,3,3,American Airlines Inc.,"Hayden, CO: Yampa Valley","Hayden, CO",CO,Colorado,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...","Dallas/Fort Worth, TX",...,1944,139.0,167.0,105.0,769.0,0,17,28,0,1516
836300,2024,6,23,PSA Airlines Inc.,"Hartford, CT: Bradley International","Hartford, CT",CT,Connecticut,"Washington, DC: Ronald Reagan Washington National","Washington, DC",...,1245,92.0,78.0,64.0,313.0,51,0,0,0,1319
998482,2024,7,20,Envoy Air,"Pensacola, FL: Pensacola International","Pensacola, FL",FL,Florida,"Chicago, IL: Chicago O'Hare International","Chicago, IL",...,213,152.0,136.0,113.0,794.0,5,0,0,0,2191
899588,2024,7,6,American Airlines Inc.,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...","Atlanta, GA",GA,Georgia,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...","Dallas/Fort Worth, TX",...,145,143.0,130.0,104.0,731.0,112,0,0,0,2129
1434377,2024,11,11,American Airlines Inc.,"Kona, HI: Ellison Onizuka Kona International a...","Kona, HI",HI,Hawaii,"Los Angeles, CA: Los Angeles International","Los Angeles, CA",...,2302,322.0,314.0,289.0,2504.0,0,0,0,0,1398


In [80]:
df.loc[df.groupby('DEST_STATE_ABR')['CARRIER_DELAY'].idxmax()] 

,YEAR,MONTH,DAY_OF_MONTH,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_NAME,DEST_CITY_NAME,...,WHEELS_ON,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
36596,2023,12,15,Delta Air Lines Inc.,"Minneapolis, MN: Minneapolis-St Paul Internati...","Minneapolis, MN",MN,Minnesota,"Anchorage, AK: Ted Stevens Anchorage Internati...","Anchorage, AK",...,1209,379.0,379.0,353.0,2519.0,1012,0,0,0,0
920338,2024,7,9,SkyWest Airlines Inc.,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...","Atlanta, GA",GA,Georgia,"Dothan, AL: Dothan Regional","Dothan, AL",...,1301,61.0,53.0,33.0,170.0,967,0,0,0,0
494273,2024,4,12,Mesa Airlines Inc.,"Denver, CO: Denver International","Denver, CO",CO,Colorado,"Little Rock, AR: Bill and Hillary Clinton Nat ...","Little Rock, AR",...,1036,128.0,119.0,94.0,771.0,1089,0,0,0,0
471469,2024,4,8,American Airlines Inc.,"San Diego, CA: San Diego International","San Diego, CA",CA,California,"Phoenix, AZ: Phoenix Sky Harbor International","Phoenix, AZ",...,2151,84.0,90.0,56.0,304.0,3300,0,6,0,48
663626,2024,5,21,American Airlines Inc.,"San Francisco, CA: San Francisco International","San Francisco, CA",CA,California,"Los Angeles, CA: Los Angeles International","Los Angeles, CA",...,1833,88.0,88.0,58.0,337.0,2043,0,0,0,0
1268044,2024,9,16,United Air Lines Inc.,"Grand Junction, CO: Grand Junction Regional","Grand Junction, CO",CO,Colorado,"Denver, CO: Denver International","Denver, CO",...,1518,76.0,63.0,42.0,212.0,1485,0,0,0,0
123198,2024,1,8,Frontier Airlines Inc.,"San Juan, PR: Luis Munoz Marin International","San Juan, PR",PR,Puerto Rico,"Hartford, CT: Bradley International","Hartford, CT",...,1651,234.0,228.0,194.0,1666.0,1267,0,0,0,0
895337,2024,7,5,American Airlines Inc.,"Austin, TX: Austin - Bergstrom International","Austin, TX",TX,Texas,"Miami, FL: Miami International","Miami, FL",...,1305,182.0,208.0,166.0,1103.0,3689,0,26,0,88
293748,2024,2,23,American Airlines Inc.,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...","Dallas/Fort Worth, TX",TX,Texas,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...","Atlanta, GA",...,2241,121.0,109.0,87.0,731.0,1364,0,0,0,12
261090,2024,2,12,Hawaiian Airlines Inc.,"Austin, TX: Austin - Bergstrom International","Austin, TX",TX,Texas,"Honolulu, HI: Daniel K Inouye International","Honolulu, HI",...,1950,520.0,525.0,495.0,3764.0,1734,0,0,0,0
